<a href="https://colab.research.google.com/github/POLSEAN/XTDML/blob/main/examples/03_xtdml_fd_exact.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DML for panel data: FD (exact) approach**

---

*Description*

Estimation of the structural parameter using double machine learning (DML) with partially linear regression (PLR) models in the context of panel data with fixed effects as in Clarke and Poselli(2023).

The package `XTDML` allows the estimation of the nuisance functions by machine learning methods and  the computation of the Neyman orthogonal score functions. `XTDML` is built on the CRAN package `DoubleML` (Bach et al., 2024), which uses the `mlr3` ecosystem and the `R6` package.

**References**

[1] Bach, P., Chernozhukov, V., Kurz, M. S., Spindler, M. and Klaassen, S. (2024), DoubleML - An Object-Oriented Implementation of Double Machine Learning in R, *Journal of Statistical Software*, 108(3):1-56.

[2] Chernozhukov, V., Chetverikov, D., Demirer, M., Duflo, E., Hansen, C., Newey, W., and Robins, J. (2018). Double/debiased machine learning for treatment and structural parameters. *The Econometrics Journal*, 21(1):C1-C68.

[3] Clarke, P. and Polselli, A. (2023). Double machine learning for static panel models with fixed effects. *arXiv preprint*, arXiv:2312.08174.

[4] Mundlak, Y. (1978). On the pooling of time series and cross section data. *Econometrica*, pages 69-85.

*Overview Code*

1. Installation of XTDML and other R packages
2. Loading the data
3. Data management with FD transformation
4. Set up of DML data environment
5. Set up of DML estimation environment
6. Extraction of DML estimates


### **The Installation of `XTDML` package**

The `XTDML` package can be installed following either options below:

1. **Installation directly from GitHub:**
  ```
    #install.packages("devtools")
    library(devtools)

    install_github("POLSEAN/XTDML")
    library(XTDML)
  ```
  *Note this code works **ONLY with RStudio (desktop)**, but not with online platforms such as Google Colab or Kaggle.*


2. **Download all folders in `XTDML`** from `https://github.com/POLSEAN/XTDML` pressing `<> CODE > Download ZIP`. Rename the downloaded .zip folder as `XTDML`, and upload it on Google Colab. Get the path and run the code `!unzip XTDML.zip` in Python, then change the RUNTIME to R and run
   ```
    #install.packages("devtools")
    library(devtools)

    wd = "~ your-directory/XTDML"
    devtools::load_all(wd)
   ```

For illustration purposes on Google Colab, we follow the second approach, but the first is recommended with RStudio (desktop).


**Set RUNTIME > CHANGE RUNTIME TYPE > Python 3**

The code below unzips the XTDML.zip folder that you have previously uploaded.

In [ ]:
!unzip XTDML.zip

Archive:  XTDML.zip
 extracting: XTDML/.gitignore        
  inflating: XTDML/.Rbuildignore     
  inflating: XTDML/.RData            
  inflating: XTDML/.Rhistory         
   creating: XTDML/.Rproj.user/
   creating: XTDML/.Rproj.user/22C44D20/
   creating: XTDML/.Rproj.user/22C44D20/bibliography-index/
 extracting: XTDML/.Rproj.user/22C44D20/cpp-definition-cache  
   creating: XTDML/.Rproj.user/22C44D20/ctx/
   creating: XTDML/.Rproj.user/22C44D20/explorer-cache/
   creating: XTDML/.Rproj.user/22C44D20/pcs/
  inflating: XTDML/.Rproj.user/22C44D20/pcs/files-pane.pper  
 extracting: XTDML/.Rproj.user/22C44D20/pcs/source-pane.pper  
  inflating: XTDML/.Rproj.user/22C44D20/pcs/windowlayoutstate.pper  
  inflating: XTDML/.Rproj.user/22C44D20/pcs/workbench-pane.pper  
   creating: XTDML/.Rproj.user/22C44D20/presentation/
   creating: XTDML/.Rproj.user/22C44D20/profiles-cache/
 extracting: XTDML/.Rproj.user/22C44D20/rmd-outputs  
 extracting: XTDML/.Rproj.user/22C44D20/saved_source_markers  

**From now change RUNTIME > SET RUNTIME TYPE > R**

In [ ]:
# 1. Install and import R packages
# Install packages
list.of.packages <- c("datawizard","mlr3","mlr3learners","mlr3tuning","paradox","xgboost","ranger","MLmetrics","devtools","tidyverse")
new.packages <- list.of.packages[!(list.of.packages %in% installed.packages()[,"Package"])]
if(length(new.packages)) install.packages(new.packages, repos = "http://cran.us.r-project.org")

# Load general packages
library(devtools)
library(tidyverse)
library(checkmate)
library(dplyr)    # alternative installation of the %>%
library(tibble)   # for add_column()
library(datawizard)
library(data.table)
# ML packages
library(mlr3)
library(mlr3learners)
library(rpart)
library(xgboost)
library(ranger)
# Packages for HP tuning
library(mlr3misc)
library(mlr3tuning)
library(paradox)
library(MLmetrics)

# Suppress error messages from ML packages
lgr::get_logger("bbotk")$set_threshold("warn")
lgr::get_logger("mlr3")$set_threshold("warn")

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘bitops’, ‘gtools’, ‘caTools’, ‘globals’, ‘listenv’, ‘PRROC’, ‘gplots’, ‘insight’, ‘checkmate’, ‘future’, ‘future.apply’, ‘lgr’, ‘mlbench’, ‘mlr3measures’, ‘mlr3misc’, ‘parallelly’, ‘palmerpenguins’, ‘bbotk’, ‘RcppEigen’, ‘ROCR’


Loading required package: usethis

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘data.table’


The following objects are masked from ‘package:lubridate’:

In [ ]:
# Additional package required to install XTDML (not always necessary, depends on the R version)
list.of.packages <- c("mvtnorm","clusterGeneration","readstata13")
new.packages <- list.of.packages[!(list.of.packages %in% installed.packages()[,"Package"])]
if(length(new.packages)) install.packages(new.packages, repos = "http://cran.us.r-project.org")

library(mvtnorm)
library(clusterGeneration)
library(readstata13)

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


Attaching package: ‘mvtnorm’


The following object is masked from ‘package:datawizard’:

    standardize


Loading required package: MASS


Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select




In [ ]:
# Install package
wd = "/content/XTDML"
devtools::load_all(wd)

ℹ Loading XTDML


### **The Data**

We use simulated data for DGP3 as in Clarke and Polselli (2023). We use a subsample (N=250) of the original dataset (with N=1,000,000), where each unit is observed over $T=10$ periods.

In this dataset, the nuisance functions are generated as follows

\begin{align*}
    l_0(x_{it}) & = b \, (x_{it,1}\cdot x_{it,3}) + a \, (x_{it,3}\cdot 1[x_{it,3}>0])\\
    m_0(x_{it}) & = a \, (x_{it,1}\cdot 1[x_{it,1}>0]) + b \, (x_{it,1}\cdot x_{it,3})
\end{align*}

where $a=0.25$ and $b=0.5$. The true structural effect is 0.5.


Note that the FD (exact) approach requires to use **transformed** data
* $\Delta y_{it}$ is the transformed output variable
* $\Delta d_{it}$ is the transformed treatment variable
* $\mathbf{x}_{it} = (x_{it,1}, \dots, x_{it,p}, x_{it-1,1}, \dots, x_{it-1,p})'$ are the set of $p=30$ control variables, but only $s=2$ are relevant; $x_{it-1,k}$ is the lag of variable $k$

where $\Delta y_{it} = y_{it}-y_{it-1}.$



In [ ]:
# 2. Load simulated data from GitHub
df = read.csv("https://raw.githubusercontent.com/POLSEAN/XTDML/main/data/dgp4_cre_short.csv")
names(df)
head(df)

[1] "id"    "time"  "x1"    "x2"    "x3"    "x4"    "x5"    "x6"    "x7"   
[10] "x8"    "x9"    "x10"   "x11"   "x12"   "x13"   "x14"   "x15"   "x16"  
[19] "x17"   "x18"   "x19"   "x20"   "x21"   "x22"   "x23"   "x24"   "x25"  
[28] "x26"   "x27"   "x28"   "x29"   "x30"   "d"     "m_x1"  "m_x2"  "m_x3" 
[37] "m_x4"  "m_x5"  "m_x6"  "m_x7"  "m_x8"  "m_x9"  "m_x10" "m_x11" "m_x12"
[46] "m_x13" "m_x14" "m_x15" "m_x16" "m_x17" "m_x18" "m_x19" "m_x20" "m_x21"
[55] "m_x22" "m_x23" "m_x24" "m_x25" "m_x26" "m_x27" "m_x28" "m_x29" "m_x30"
[64] "m_d"   "y"

,id,time,x1,x2,x3,x4,x5,x6,x7,x8,⋯,m_x23,m_x24,m_x25,m_x26,m_x27,m_x28,m_x29,m_x30,m_d,y
,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,1,2.2286613,1.6905339,-2.7940819,11.1143300,8.826218,-3.9921484,3.253239,6.0599380,⋯,0.9233138,-0.1381521,4.3948,-1.040279,-0.7359572,2.131381,2.381721,-1.200919,0.7686889,-3.973063
2,1,2,7.4777365,0.8922307,-2.0939696,-4.0721574,3.936116,5.9183326,1.831009,3.8444767,⋯,0.9233138,-0.1381521,4.3948,-1.040279,-0.7359572,2.131381,2.381721,-1.200919,0.7686889,-10.303778
3,1,3,-6.7781310,6.7881875,-1.8140866,2.6128931,8.065602,3.3242040,3.965238,2.8951371,⋯,0.9233138,-0.1381521,4.3948,-1.040279,-0.7359572,2.131381,2.381721,-1.200919,0.7686889,11.109933
4,1,4,-0.3668897,-3.5370691,1.8084828,-8.5668802,9.544738,11.0767380,2.877717,2.4498920,⋯,0.9233138,-0.1381521,4.3948,-1.040279,-0.7359572,2.131381,2.381721,-1.200919,0.7686889,2.730979
5,1,5,9.0546560,1.4772189,0.2424859,-0.5902931,-6.163388,0.7534147,-3.353442,-0.4494378,⋯,0.9233138,-0.1381521,4.3948,-1.040279,-0.7359572,2.131381,2.381721,-1.200919,0.7686889,2.967221
6,1,6,-5.8348851,5.7826090,-2.5719683,1.9519805,-1.261380,-0.3874538,-4.370841,7.7953877,⋯,0.9233138,-0.1381521,4.3948,-1.040279,-0.7359572,2.131381,2.381721,-1.200919,0.7686889,11.993943


In [ ]:
# 3. Transform variables
# keep variables to transform (no var means)

# Create (a) lags of X and (b) Delta_y and Delta_d
xvars = paste0("x", 1:30)

df.fd = df %>%
  group_by(id) %>%
  mutate(across(xvars, ~  lag(.x), .names = "L.{col}"))   %>%
  mutate(across(starts_with(c("d", "y")), ~ c(NA, diff(.x))))  %>%
  ungroup()

# Use complete.cases() to identify rows without missing values
complete_rows <- complete.cases(df.fd)

# Subset the data frame to keep only complete rows
df.fd <- df.fd[complete_rows, ]
df.fd = as.data.frame(df.fd)
names(df.fd)

[1] "id"    "time"  "x1"    "x2"    "x3"    "x4"    "x5"    "x6"    "x7"   
[10] "x8"    "x9"    "x10"   "x11"   "x12"   "x13"   "x14"   "x15"   "x16"  
[19] "x17"   "x18"   "x19"   "x20"   "x21"   "x22"   "x23"   "x24"   "x25"  
[28] "x26"   "x27"   "x28"   "x29"   "x30"   "d"     "m_x1"  "m_x2"  "m_x3" 
[37] "m_x4"  "m_x5"  "m_x6"  "m_x7"  "m_x8"  "m_x9"  "m_x10" "m_x11" "m_x12"
[46] "m_x13" "m_x14" "m_x15" "m_x16" "m_x17" "m_x18" "m_x19" "m_x20" "m_x21"
[55] "m_x22" "m_x23" "m_x24" "m_x25" "m_x26" "m_x27" "m_x28" "m_x29" "m_x30"
[64] "m_d"   "y"     "L.x1"  "L.x2"  "L.x3"  "L.x4"  "L.x5"  "L.x6"  "L.x7" 
[73] "L.x8"  "L.x9"  "L.x10" "L.x11" "L.x12" "L.x13" "L.x14" "L.x15" "L.x16"
[82] "L.x17" "L.x18" "L.x19" "L.x20" "L.x21" "L.x22" "L.x23" "L.x24" "L.x25"
[91] "L.x26" "L.x27" "L.x28" "L.x29" "L.x30"

## **Estimation and inference with DML for FD**

The section below consists in setting up the DML data and estimation environments, and proceed with the actual estimation.

### **4. Set up DML data environment**
Initalization of `dml_approx_data`  from `data.frame`. Arguments to pass:

```
dml_approx_data_from_data_frame(data,
                  x_cols = NULL,
                  y_col = NULL,
                  d_cols = NULL,
                  z_cols = NULL,
                  cluster_cols = NULL
                  )

```       

In [ ]:
# 4. Set up DML data environment

# get the names of the variables
x_cols  = paste0("x", 1:30)
Lx_cols = paste0("L.x", 1:30)
xvars = c(x_cols, Lx_cols)

# set up data for DML procedure
obj_dml_data = dml_approx_data_from_data_frame(df.fd,
                            x_cols = xvars,  y_col = "y", d_cols = "d",
                            cluster_cols = "id")
obj_dml_data$print()

================= DML-FE Object ==================


------------------ Data summary ------------------
Outcome variable: y
Treatment variable(s): d
Cluster variable(s): id
Covariates: x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17, x18, x19, x20, x21, x22, x23, x24, x25, x26, x27, x28, x29, x30, L.x1, L.x2, L.x3, L.x4, L.x5, L.x6, L.x7, L.x8, L.x9, L.x10, L.x11, L.x12, L.x13, L.x14, L.x15, L.x16, L.x17, L.x18, L.x19, L.x20, L.x21, L.x22, L.x23, L.x24, L.x25, L.x26, L.x27, L.x28, L.x29, L.x30
Instrument(s): 
No. Observations: 2250


### **5. Set up DML estimation environment**

Arguments to pass in `dml_approx_plr` function that Creates a new instance of this R6 class.

```
 dml_approx_plr$new(data,
      ml_l,
      ml_m,
      ml_g = NULL,
      n_folds = 5,
      n_rep = 1,
      score = "orth-PO",               # or "orth-IV"
      dml_procedure = "dml2",          # or "dml1"
      draw_sample_splitting = TRUE,
      apply_cross_fitting = TRUE
      )

```

In [ ]:
# 5. Set up DML estimation environment
set.seed(1408)
learner = lrn("regr.rpart")
ml_l = learner$clone()
ml_m = learner$clone()

dml_rpart = dml_approx_plr$new(obj_dml_data,
                               ml_l = ml_l, ml_m = ml_m)

# set up a list of parameter grids
param_grid = list("ml_l" = ps(cp = p_dbl(lower = 0.01, upper = 0.02),
                              maxdepth = p_int(lower = 2, upper = 10)),
                  "ml_m" = ps(cp = p_dbl(lower = 0.01, upper = 0.02),
                              maxdepth = p_int(lower = 2, upper = 10)))

tune_settings = list(terminator = mlr3tuning::trm("evals", n_evals = 10),
                      algorithm = tnr("grid_search"), resolution = 20)

dml_rpart$tune(param_set = param_grid, tune_settings = tune_settings)

# Estimate target/causal parameter
dml_rpart$fit()
dml_rpart$print()
print(dml_rpart$params)

TuningInstanceSingleCrit is deprecated. Use TuningInstanceBatchSingleCrit instead.

TuningInstanceSingleCrit is deprecated. Use TuningInstanceBatchSingleCrit instead.



[1] "rmses in fold 1 : 38.2667513392526"
[1] "theta_subsample_mean in fold 1: 1.22415427924618"
[1] "rmses in fold 2 : 49.6052779583265"
[1] "theta_subsample_mean in fold 2: 0.974890246309583"
[1] "rmses in fold 3 : 30.8895024383384"
[1] "theta_subsample_mean in fold 3: 1.10222928493362"
[1] "rmses in fold 4 : 26.5752469964114"
[1] "theta_subsample_mean in fold 4: 1.16282615119212"
[1] "rmses in fold 5 : 44.0157121979217"
[1] "theta_subsample_mean in fold 5: 1.21463747381288"
[1] "theta in dml2: 1.21463747381288"
[1] "rmse in dml2: 44.0157121979217"
================= dml_approx_plr Object ==================



------------------ Data summary ------------------
Outcome variable: y
Treatment variable: d
Covariates: x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17, x18, x19, x20, x21, x22, x23, x24, x25, x26, x27, x28, x29, x30, L.x1, L.x2, L.x3, L.x4, L.x5, L.x6, L.x7, L.x8, L.x9, L.x10, L.x11, L.x12, L.x13, L.x14, L.x15, L.x16, L.x17, L.x18, L.x19, L.x20, L.x21

In [ ]:
# Display table with results
library(xtable)

table = matrix(0, 1, 6)
table[1,] = cbind(dml_rpart$coef_theta,dml_rpart$se_theta,dml_rpart$pval_theta,dml_rpart$model_rmse,as.numeric(dml_rpart$rmses["ml_l"]),as.numeric(dml_rpart$rmses["ml_m"]))

colnames(table)= c("Estimate", "Std. Error", "P-value", "Model RMSE", "MSE of l", "MSE of m")
rownames(table)= c("DML-CART")
tab = xtable(table)
tab

,Estimate,Std. Error,P-value,Model RMSE,MSE of l,MSE of m
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
DML-CART,1.214637,0.03714296,1.47784e-234,44.01571,20.17328,12.65226
